In [1]:
%load_ext lab_black

In [49]:
import ee
import pandas as pd
import folium
import matplotlib.pyplot as plt

ee.Initialize()

In [50]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict["tile_fetcher"].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True,
    ).add_to(self)


folium.Map.add_ee_layer = add_ee_layer

In [51]:
# get states of India as a feature collection
states = ee.FeatureCollection("users/mdm/india_soiStates")
# districts = ee.FeatureCollection('users/mdm/za/za_districts')

In [52]:
firms = (
    ee.ImageCollection("FIRMS")
    .filterDate("2021-08-01", "2021-10-30")
    .select("T21", "confidence", "line_number")
    .mosaic()
)

#'properties.T21', 'properties.confidence', 'properties.line_number'
# firms.getInfo()["bands"]

In [58]:
# Define a map centered on India
mapa = folium.Map(location=[22.369085, 79.893286], zoom_start=10)

fireVis = {
    "bands": ["T21"],
    "gamma": 0.95,
    "min": 300.0,
    "max": 509.29,
}
# Add the image layer to the map and display it.
mapa.add_ee_layer(firms, fireVis, "FIRMS")

In [59]:
mapa

# Calculate fires per state

In [66]:
# Create a composite image with firms as
composite = ee.Image.cat([firms])

In [69]:
dist_stats = (
    composite.reduceRegions(collection=states, reducer="count", scale=500)
    .select("T21", "confidence", "state")
    .getInfo()
)

In [70]:
# Convert dictionary into pandas dataframe
dtstats_df = pd.DataFrame()
for dist in dist_stats["features"]:
    df = pd.DataFrame([dist["properties"]], columns=dist["properties"].keys())
    dtstats_df = pd.concat([dtstats_df, df], axis=0)

In [40]:
dtstats_df

,T21,confidence,state
0,0,0,ANDAMAN & NICOBAR
0,0,0,MIZORAM
0,20,20,ARUNACHAL PRADESH
0,0,0,NAGALAND
0,12,7,ASSAM
0,0,0,MEGHALAYA
0,1,1,MANIPUR
0,0,0,TRIPURA
0,8,7,JAMMU & KASHMIR
0,0,0,HIMACHAL PRADESH


# Extract all fires

In [81]:
print(firms.reduceToVectors())

ee.FeatureCollection({
  "functionInvocationValue": {
    "functionName": "Image.reduceToVectors",
    "arguments": {
      "image": {
        "functionInvocationValue": {
          "functionName": "ImageCollection.mosaic",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.map",
                "arguments": {
                  "baseAlgorithm": {
                    "functionDefinitionValue": {
                      "argumentNames": [
                        "_MAPPING_VAR_0_0"
                      ],
                      "body": {
                        "functionInvocationValue": {
                          "functionName": "Image.select",
                          "arguments": {
                            "bandSelectors": {
                              "constantValue": [
                                "T21",
                                "confidence",
                                "line_n

In [76]:
ee.batch.Export.image.toDrive(firms)

<Task EXPORT_IMAGE: myExportImageTask (UNSUBMITTED)>

# Extracted
https://colab.research.google.com/drive/1F5dDDTVC3QU2zYn-p7GjC-JoPp4IiE11#scrollTo=7gvMqzKNMsaf
